In [ ]:
!pip install datasets==1.11.0
!pip install transformers==4.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 8.2 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 212 kB 65.5 MB/s 
     |████████████████████████████████| 132 kB 62.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 56.6 MB/s 
     |████████████████████████████████| 880 kB 64.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=123636cb21085c07d28ecc2bc5a84f12e8c573a613e48a23c62255e577079be1
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
import datasets
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, get_cosine_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import pickle
import random
import os
import numpy as np
import pandas as pd
import plotly.express as px

COLAB = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/UAS PBA/Dataset/

/content/drive/My Drive/UAS PBA/Dataset


In [ ]:
!pip install accelerate==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 3.9 MB/s 


In [ ]:
from accelerate import Accelerator

In [ ]:
# load dataset concatanated by 'train' and 'validation'
train_dataset = datasets.load_dataset('squad', split='train')
test_dataset = datasets.load_dataset('squad', split='validation') 
dataset = datasets.concatenate_datasets([train_dataset, test_dataset])


# Using automatic mixed precision training (fp16)
fp16_training = True
if fp16_training:
    accelerator = Accelerator(fp16=True)
    device = accelerator.device
elif torch.cuda.is_available():
      device = 'cuda'
else:
      device = 'cpu'

# load pretrained model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-cased').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# The answer of some cases don't exist in their context.
def check_target_exist(input_ids, target_tokenized):
    check_list = []
    for i in range(0, (len(input_ids)-len(target_tokenized)+1)):
        window = input_ids[i:i+len(target_tokenized)]
        if window == target_tokenized:
            check_list.append('V')
    return 'V' in check_list

clean_case = [i for i in tqdm(range(0, len(dataset))) 
                    if check_target_exist(tokenizer.encode(dataset[i]['context'], add_special_tokens=False),
                                          tokenizer.encode(dataset[i]['answers']['text'][0], add_special_tokens=False))]

  0%|          | 0/98169 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# split dataset into train_dataset, dev_dataset, test_dataset with size of 8:1:1
clean_dataset = dataset.select(clean_case)
dataset = clean_dataset.train_test_split(test_size = 0.1)
train_dev_dataset = dataset['train']
train_dev_dataset = train_dev_dataset.train_test_split(test_size = 0.111)
train_dataset, dev_dataset, test_dataset = train_dev_dataset['train'], train_dev_dataset['test'], dataset['test']

In [ ]:
# Tokenize questions and paragraphs separately
train_questions_tokenized = tokenizer([case['question'] for case in tqdm(train_dataset)], add_special_tokens=False)
dev_questions_tokenized = tokenizer([case['question'] for case in tqdm(dev_dataset)], add_special_tokens=False)
test_questions_tokenized = tokenizer([case['question'] for case in tqdm(test_dataset)], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer([case['context'] for case in tqdm(train_dataset)], add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer([case['context'] for case in tqdm(dev_dataset)], add_special_tokens=False)
test_paragraphs_tokenized = tokenizer([case['context'] for case in tqdm(test_dataset)], add_special_tokens=False)

  0%|          | 0/78377 [00:00<?, ?it/s]

  0%|          | 0/9787 [00:00<?, ?it/s]

  0%|          | 0/9796 [00:00<?, ?it/s]

  0%|          | 0/78377 [00:00<?, ?it/s]

  0%|          | 0/9787 [00:00<?, ?it/s]

  0%|          | 0/9796 [00:00<?, ?it/s]

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, tokenizer, dataset, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.tokenized_questions = tokenized_questions['input_ids']
        self.tokenized_paragraphs = tokenized_paragraphs['input_ids']
        self.max_question_len = 30
        self.max_paragraph_len = 250
        self.doc_stride = 150
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

        
    def __len__(self):
        return len(self.dataset)

    
    def padding(self, input_ids_question, input_ids_paragraph):
        # pad zero such that every sequence of length is max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        input_ids = input_ids_question + input_ids_paragraph + [0]* padding_len
        token_type_ids = [0]* len(input_ids_question) + [1]* len(input_ids_paragraph) + [0]* padding_len
        attention_mask = [1]* (len(input_ids_question) + len(input_ids_paragraph)) + [0]* padding_len
        return input_ids, token_type_ids, attention_mask
    
    
    def locate_start_end_target_token(self, input_ids, target_tokenized):
        # locate the indice of target start token and target end token
        window_size = len(target_tokenized)
        target_window = []
        for i in range(0, len(input_ids)-window_size+1):
            search_window = input_ids[i: i+window_size]
            if search_window == target_tokenized:
                target_window.append((i, i+window_size-1))
        target_start_token, target_end_token = target_window[0][0], target_window[0][1]
        return target_start_token, target_end_token
    
    
    def check_target_exist(self, input_ids, target_tokenized):
        check_list = []
        for i in range(0, (len(input_ids)-len(target_tokenized)+1)):
            if input_ids[i:i+len(target_tokenized)] == target_tokenized:
                check_list.append('V')
        return 'V' in check_list

    
    def __getitem__(self, index):
        question = self.dataset[index]
        tokenized_question = self.tokenized_questions[index]
        tokenized_paragraph = self.tokenized_paragraphs[index]
        answer = question['answers']['text'][0]
        answer_tokenized = self.tokenizer.encode(answer, add_special_tokens=False) # add_special_tokens=False means that embedding without [CLS] & [SEP]

        if self.split == 'train':
            # output "designated window", which contain answer token
            windows = []
            for i in range(0, max(1, len(tokenized_paragraph)-self.max_paragraph_len+1)):
                windows.append(tokenized_paragraph[i:i+self.max_paragraph_len])
            windows_contain_answer = [x for x in windows if self.check_target_exist(x, answer_tokenized)]
            random.seed(index)
            designated_window = windows_contain_answer[random.randint(0, len(windows_contain_answer)-1)]
            
            # add special tokens ([CLS]->[101], [SEP]->[102]) to tokenized_question
            # add special tokens ([SEP]->[102]) to tokenized_paragraph
            input_ids_question = [101] + tokenized_question[0:self.max_question_len] + [102]
            input_ids_paragraph = designated_window + [102]
            
            # padding zero such that every sequence is equal to "self.max_seq_len"
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            answer_start_token, answer_end_token = self.locate_start_end_target_token(input_ids, answer_tokenized)
            
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, which length of every window is "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # add special tokens ([CLS]->[101], [SEP]->[102]) to tokenized_question
                # add special tokens ([SEP]->[102]) to tokenized_paragraph
                input_ids_question = [101] + tokenized_question[0:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph[i: i + self.max_paragraph_len] + [102]
                
                # padding zero such that every sequence is equal to "self.max_seq_len"
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)



train_set = QA_Dataset("train", tokenizer, train_dataset, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", tokenizer, dev_dataset, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", tokenizer, test_dataset, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 16

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [ ]:
# Obtain the answer

def evaluate(data, output):    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain the index by maximum logits in 'start_logits' & 'end_logits'
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous window
        if prob > max_prob:
            max_prob = prob
            
            # exchange start_index and end_index if start_index > end_index
            # Is it reasonable? (TODO)
            if start_index > end_index:
                start_index, end_index = end_index, start_index
            
            # Convert tokens to chars
            # ex: [16416, 1158, 1104, 1103, 5568, 2813] --> 'partitioning of the Ottoman Empire'
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    return answer

In [ ]:
# Training Model

num_epoch = 10
validation = True
logging_step = 100
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=30, num_training_steps=len(train_loader))


if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_index, end_index
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        # output.start_logits = output.end_logits = (batch_size, sequence_length)
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        step += 1

        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_dataset[i]['answers']['text'][0]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()
        
    scheduler.step()



Start Training ...


/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  0%|          | 0/4899 [00:00<?, ?it/s]

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save_pretrained("/content/drive/My Drive/UAS PBA/Dataset/")

In [ ]:
model.from_pretrained("/content/drive/My Drive/UAS PBA/Dataset/")

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_